In [1]:
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7


Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 61.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 88.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 75.5 MB/s eta 0:00:00


In [3]:
# pip3 uninstall -y transformers
# pip3 install git+https://github.com/huggingface/transformers.git
!pip install torch


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
model_name_or_path = "TheBloke/Llama-2-70B-chat-GPTQ"

#! Check the number of available GPUs and prepare a device map
num_gpus = torch.cuda.device_count()
device_map = {i: f'cuda:{i}' for i in range(num_gpus)}



# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
print("READY!!!!!!!!!!!")

config.json:   0%|          | 0.00/840 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/35.3G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]



*** Generate:
<s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Tell me about AI[/INST]

Artificial intelligence (AI) refers to the ability of machines or computer programs to mimic intelligent human behavior, such as learning, problem-solving, decision-making, and perception. AI can be applied in various fields, including natural language processing, image and speech recognition, robotics, healthcare, finance, and transportation.

There are several types of AI, including:

1. Narrow or weak AI: This type o

In [12]:
i = 0
print(num_gpus)
print(device_map)

4
{0: 'cuda:0', 1: 'cuda:1', 2: 'cuda:2', 3: 'cuda:3'}


In [18]:

prompt = "Generate Dataset: generate about 200 sets of answers and question and 100 statements"

instruction = """
Are you to return random generated facts that are fed into an Large Lanugage Model(LLM) Dataset. it should contain information such as
news, historical information, literature, facts about science, maths and other relavent human information such as conversation. 
Type of generation can be in a form of question, answer, statement and other form.
Return around 1 sentence to a paragraph of information
please don't share false information. Generate them random make sure they are not the same, this is a dataset after all. ."""
prompt_template=f'''[INST] <<SYS>>
{instruction}
<</SYS>>
{prompt}[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)
x = pipe(prompt_template)
y = x[0]['generated_text']
i +=1



*** Generate:
<s> [INST] <<SYS>>

Are you to return random generated facts that are fed into an Large Lanugage Model(LLM) Dataset. it should contain information such as
news, historical information, literature, facts about science, maths and other relavent human information such as conversation. 
Type of generation can be in a form of question, answer, statement and other form.
Return around 1 sentence to a paragraph of information
please don't share false information. Generate them random make sure they are not the same, this is a dataset after all. .
<</SYS>>
Generate Dataset: generate about 200 sets of answers and question and 100 statements[/INST]

I can certainly help you generate a dataset of random facts for your large language model (LLM) dataset. Here are 200 sets of answers and questions, as well as 100 statements that contain information on various topics such as news, history, literature, science, math, and other relevant human information.

**Sets of Answers and Question

In [19]:
x

[{'generated_text': '[INST] <<SYS>>\n\nAre you to return random generated facts that are fed into an Large Lanugage Model(LLM) Dataset. it should contain information such as\nnews, historical information, literature, facts about science, maths and other relavent human information such as conversation. \nType of generation can be in a form of question, answer, statement and other form.\nReturn around 1 sentence to a paragraph of information\nplease don\'t share false information. Generate them random make sure they are not the same, this is a dataset after all. .\n<</SYS>>\nGenerate Dataset: generate about 200 sets of answers and question and 100 statements[/INST]\n\nI understand that you want me to generate a dataset containing 200 sets of questions and answers, as well as 100 statements. The dataset should include information such as news, historical information, literature, facts about science, math, and other relevant human information, similar to what a large language model would b

In [20]:
y

'[INST] <<SYS>>\n\nAre you to return random generated facts that are fed into an Large Lanugage Model(LLM) Dataset. it should contain information such as\nnews, historical information, literature, facts about science, maths and other relavent human information such as conversation. \nType of generation can be in a form of question, answer, statement and other form.\nReturn around 1 sentence to a paragraph of information\nplease don\'t share false information. Generate them random make sure they are not the same, this is a dataset after all. .\n<</SYS>>\nGenerate Dataset: generate about 200 sets of answers and question and 100 statements[/INST]\n\nI understand that you want me to generate a dataset containing 200 sets of questions and answers, as well as 100 statements. The dataset should include information such as news, historical information, literature, facts about science, math, and other relevant human information, similar to what a large language model would be trained on.\n\nHer

In [16]:
with open(f"70B_out{i}.txt","w") as f:
    f.write(x)